# 異常生成スクリプト

下記スクリプトは正常データのCSVファイルから異常データを生成します。
正常データの一部を、倍率、ノイズ、突発ピークを付与することができます。
程度は数値を調整することで加減ができます。
また、どこに異常値を与えるか指定することも可能です。
詳しい変数調整内容はスクリプト後に掲載します。

In [1]:
# 最終版

import pandas as pd
import numpy as np
from pathlib import Path
import os

# ===== 設定 =====
col_index_current = 4  # 電流値
col_index_label = 6    # ラベル列（7列目 = index 6）
skip_header_rows = 31
output_dir = "generated_anomaly"
os.makedirs(output_dir, exist_ok=True)

# 区間を明示的に指定（サンプル）
anomaly_windows = [(6600, 7700)]

# ===== 疑似異常生成関数 =====
def generate_synthetic_anomaly(data, window_indices, scale=1.1, noise_level=0.02, peak=False):
    new_data = data.copy()
    labels = np.zeros_like(data, dtype=int)
    for start, end in window_indices:
        new_data[start:end] = new_data[start:end] * scale
        noise = (np.random.rand(end - start) - 0.5) * 2 * noise_level * np.abs(new_data[start:end])
        new_data[start:end] += noise
        if peak:
            peak_idx = np.random.randint(start, end)
            new_data[peak_idx] += np.max(np.abs(new_data[start:end])) * 0.5
        labels[start:end] = 1
    return new_data, labels

# ===== メイン処理 =====
def create_synthetic_abnormal_csv(input_csv, output_csv, anomaly_windows=None):
    """
    anomaly_windows: list of (start, end) サンプルインデックスで指定（0始まり, endは非包含）
    """
    # 保存先ディレクトリを用意
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    # CSV読み込み
    df = pd.read_csv(input_csv, header=None, encoding='cp932', sep=None, engine='python', skiprows=skip_header_rows)

    # 元の電流値
    current = df.iloc[:, col_index_current].to_numpy().astype(float)
    total_len = len(current)

    # ---- ラベル列がなければ新規追加、あれば初期化 ----
    if df.shape[1] <= col_index_label:
        for _ in range(col_index_label + 1 - df.shape[1]):
            df[df.shape[1]] = 0
    else:
        df.iloc[:, col_index_label] = 0

    # ---- 区間の解決（指定が無ければランダム） ----
    if anomaly_windows is None:
        print("[INFO] anomaly_windows 未指定 → ランダム生成します。")
        n_windows = 3
        window_size = max(1, total_len // 20)
        anomaly_windows = []
        for _ in range(n_windows):
            start = np.random.randint(0, max(1, total_len - window_size))
            anomaly_windows.append((start, start + window_size))
    else:
        # バリデーション＆クリップ
        validated = []
        for (s, e) in anomaly_windows:
            s = int(max(0, s))
            e = int(min(total_len, e))
            if e - s >= 2:
                validated.append((s, e))
        if not validated:
            print("[WARN] 指定した anomaly_windows が無効だったため、ランダム生成に切替えます。")
            n_windows = 3
            window_size = max(1, total_len // 20)
            for _ in range(n_windows):
                start = np.random.randint(0, max(1, total_len - window_size))
                validated.append((start, start + window_size))
        anomaly_windows = validated
        print(f"[INFO] 使用する異常区間: {anomaly_windows}")

    # 疑似異常生成
    new_current, labels = generate_synthetic_anomaly(
        current, anomaly_windows, scale=1.3, noise_level=0.05, peak=True
    )

    # データフレームに反映
    df.iloc[:, col_index_current] = new_current
    df.iloc[:, col_index_label] = labels

    # 保存
    df.to_csv(output_csv, index=False, header=False, encoding='cp932')
    print(f"[INFO] 疑似異常CSV保存 → {output_csv}")

# ===== 実行例 =====
input_csv = r"C:/Users/000337/Desktop/report/plactice/PoC plactice/デジタルオシロ/GS/GS0822/edit/100-110-120SPM_07.csv"
output_csv = r"C:/Users/000337/Desktop/report/plactice/PoC plactice/デジタルオシロ/GS/GS0822/anomaly/100-110-120SPM_07_synthetic.csv"

# ★ ここで anomaly_windows を必ず渡す！
create_synthetic_abnormal_csv(input_csv, output_csv, anomaly_windows=anomaly_windows)


[INFO] 使用する異常区間: [(6600, 7700)]
[INFO] 疑似異常CSV保存 → C:/Users/000337/Desktop/report/plactice/PoC plactice/デジタルオシロ/GS/GS0822/anomaly/100-110-120SPM_07_synthetic.csv


### 異常生成スクリプトで調整できる変数（scale, noise_level, peak）は、それぞれ現場で起こり得る異常の種類に対応させることができます。				
### 以下に、変数ごとに「どの数値を設定すべきか」と「現場で想定される異常の意味付け」を整理しました。				
				
				
				
## ① scale（倍率）							
				
- 意味: 電流値をどれくらい増加させるか。				
				
- 小さい値 (1.1〜1.2) 
   → 軽微な摩耗、潤滑不良の初期、軽い負荷変動。				
				
- 中程度 (1.3〜1.5)				
   → 軸受の摩耗進行、ベルトの張り過ぎ、加工物のばらつきによる負荷増加。				
				
- 大きい値 (1.6以上)				
   → 部品の固着、負荷急増（詰まりや異物噛み込み）、重大な異常直前。				
				
- 指標: 実際の現場でも、正常比で「10〜30%増し」くらいはよく見られる初期異常。
　　　  50%以上はかなり深刻な状態を模擬できます。				
				
				
				
## ② noise_level（ノイズ強度）						
				
- 意味: ±何%のランダムな揺らぎを入れるか。				
				
- 小さい値 (0.01〜0.03)				
   → 通常のバラツキに近い微振動。モータの小さなアンバランスや電源リップル。				
				
- 中程度 (0.05〜0.1)				
   → 軸受けが傷んできた時のガタ、負荷変動の影響。				
				
- 大きい値 (0.15以上)				
   → 機械的なガタつきや、電気ノイズ（インバータ不具合、ケーブル接触不良）。				
				
- 指標: 現場では 「正常±数%」が標準的揺らぎ。
　　　  10%以上の揺らぎは異常として扱いやすいです。				
				
				
				
## ③ peak（突発ピーク）				
				
- 意味: 区間のどこかに突発的な電流ピークを入れる。				
				
- OFF (False)				
   → じわじわ進行する異常を模擬（摩耗・負荷増加）。				
				
- ON (True)				
   → 突発的な衝撃を模擬（異物噛み込み、瞬間的な過負荷、チャッタリング）。				
				
- 指標: 製造ラインでは「異物混入」「加工条件不良」などで一瞬だけピークが出ることがあります。
　  　　再現すると検知モデルの感度確認に役立ちます。				
				
				
				
### まとめ（推奨組み合わせ例）				
				
 
-  軽微な摩耗　scale：1.1　 noise_level：0.02　 peak：FALSE ＝ 軸受初期摩耗、潤滑不足
-  負荷増加　　scale：1.3　 noise_level：0.05 　peak：FALSE ＝ ベルト張り過ぎ、負荷変動|
-  ガタ・振動　scale：1.2　 noise_level：0.1　　peak：FALSE ＝ 軸受ガタ、部品の緩み    
-  突発異物　　scale：1.5　 noise_level：0.05　	peak：TRUE	 ＝ 異物噛み込み、加工不良  
-  深刻異常　　scale：1.8　 noise_level：0.15　	peak：TRUE	 ＝ 固着、部品破損寸前      
								
### ◆ ポイント ◆
				
- scale = 負荷の増加具合				
				
- noise_level = 振動・ばらつき具合				
				
- peak = 瞬間的な異常有無				
				
この3つを変えることで、現場シナリオに沿った「疑似異常データセット」をいくつも作ることができます。				
